```perl
#!/usr/bin/perl

use warnings;
use strict;
use utf8;

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");

my $title = '';
my $id = '';
while(<>) {
	chomp;
	if(m!<title>([^<]+)</title>!) {
		$title = $1;
	}
        if(m!^      <id>([^<]+)</id>!) {
		$id = $1;
	}
	if(m!{{IPA\|ga!) {
		if($title and $title ne '') {
			print "\n---\n";
			print "$title\n";
			print "ID $id\n";
			$title = '';
			$id = '';
		}
		print "$_\n";
	} elsif(m!Quiggin!) {
		print "$_\n";
	}
}
```

In [38]:
INPUT = "/Users/joregan/Playing/irish-gists/wiktionary/2025-extract.txt"

In [39]:
with open(INPUT) as f:
    text = f.read()
pieces = text.split("\n---\n")

In [40]:
pieces = [p.strip() for p in pieces if p != ""]

In [41]:
pieces = [p for p in pieces if "|ga|" in p]

In [42]:
quiggin = [p for p in pieces if "Quiggin" in p]

In [57]:
import urllib.parse

def make_link(word, id):
    url_word = urllib.parse.quote(word.replace(" ", "_"))
    return f"https://en.wiktionary.org/w/index.php?title={url_word}&oldid={id}"

def clean_line(line):
    if line.startswith("* "):
        line = line[2:]
    if "</text>" in line:
        line = line.replace("</text>", "")
    return line.strip()

def extract_entry(p):
    lines = p.split("\n")
    title = lines[0].strip()
    id = lines[1].strip().replace("ID ", "")
    lines = lines[2:]
    if len(lines) == 1:
        if "Quiggin" in lines[0]:
            return {
                "title": title,
                "id": id,
                "url": make_link(title, id),
                "raw": clean_line(lines[0])
            }
    # hey copilot, it's not a quote, stop suggesting that
    # it's not a definition either
    elif len(lines) == 2:
        if "* {{R:ga:Quiggin" in lines[1]:
            return {
                "title": title,
                "id": id,
                "url": make_link(title, id),
                "raw": [clean_line(p) for p in lines],
            }
    else:
        def yes(line):
            return "Quiggin" in line or "Ulster" in line
        lines = [clean_line(p) for p in lines if yes(p)]
        if lines != []:
            return {
                "title": title,
                "id": id,
                "url": make_link(title, id),
                "raw": lines,
            }
    return None

In [43]:
sep_quote = [p for p in quiggin if "* {{R:ga:Quiggin" in p]

In [58]:
extr = [extract_entry(p) for p in pieces]
extr = [e for e in extr if e is not None]

In [59]:
with open("wiktionary-quiggin.json", "w") as f:
    import json
    json.dump(extr, f, indent=2)